In [1]:
#Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import syft as sy
import sys
import pdb 
import math
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import torchvision.models as models
from random import shuffle
from torch.utils.data import SubsetRandomSampler, WeightedRandomSampler
import torchvision.models as models
import datetime
import torch.nn.init as init
import torch

W0304 07:47:43.830301 16004 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'c:\users\florian\appdata\local\programs\python\python37\lib\site-packages\tf_encrypted-0.5.9-py3.7.egg\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0-rc3.so'
W0304 07:47:43.861301 16004 module_wrapper.py:139] From c:\users\florian\appdata\local\programs\python\python37\lib\site-packages\tf_encrypted-0.5.9-py3.7.egg\tf_encrypted\session.py:24: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)
use_cuda = True
kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 512
np.random.seed(1)
torch.manual_seed(1)
torch.set_num_threads(4)
backdoorType = "backdoor_green_1_percent"

# Creating workers

In [3]:
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning

alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker 
chalie = sy.VirtualWorker(hook, id="chalie")  # <-- NEW: define remote worker
dave = sy.VirtualWorker(hook, id="dave")  # <-- NEW: define remote worker
evelyn = sy.VirtualWorker(hook, id="evelyn")  # <-- NEW: define remote worker
a6= sy.VirtualWorker(hook, id="a6")  # <-- NEW: define remote worker
a7= sy.VirtualWorker(hook, id="a7")  # <-- NEW: define remote worker
a8= sy.VirtualWorker(hook, id="a8")  # <-- NEW: define remote worker
a9= sy.VirtualWorker(hook, id="a9")  # <-- NEW: define remote worker
#a10= sy.VirtualWorker(hook, id="a10")  # <-- NEW: define remote worker
#a11= sy.VirtualWorker(hook, id="a11")  # <-- NEW: define remote worker
#a12= sy.VirtualWorker(hook, id="a12")  # <-- NEW: define remote worker
#a13= sy.VirtualWorker(hook, id="a13")  # <-- NEW: define remote worker
#a14= sy.VirtualWorker(hook, id="a14")  # <-- NEW: define remote worker
#a15= sy.VirtualWorker(hook, id="a15")  # <-- NEW: define remote worker
#a16= sy.VirtualWorker(hook, id="a16")  # <-- NEW: define remote worker
#a17= sy.VirtualWorker(hook, id="a17")  # <-- NEW: define remote worker
#a18= sy.VirtualWorker(hook, id="a18")  # <-- NEW: define remote worker
#a19= sy.VirtualWorker(hook, id="a19")  # <-- NEW: define remote worker


fraudulin = sy.VirtualWorker(hook, id="fraudulin")
#fraudrich = sy.VirtualWorker(hook, id="fraudrich") 

compute_nodes = [alice, bob, chalie, dave, evelyn, a6, a7, a8, a9]
frauds = [fraudulin]

# Helper function for dataset loader generation

In [4]:
#def generateLoadersPerClass(dataset):
#    #loaders per class
#    loaders_per_class = []
#    for class_name in dataset.classes:
#        # get the indices in the dataset that are relative to that class
#        idx = [
#            pos for pos, item in enumerate(dataset.samples)
#            if item[1] == dataset.class_to_idx[class_name]]
#        # construct the corresponding dataloader thanks to a SubsetRandomSampler
#        loaders_per_class += [torch.utils.data.DataLoader(
#            dataset, 
#            batch_size=batch_size,
#            sampler=SubsetRandomSampler(idx),
#            **kwargs)]
#    return loaders_per_class

# Loading training & test datasets

In [5]:
data_transform = transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),                     
        transforms.Normalize(                     
            mean=[0.485, 0.456, 0.406],               
            std=[0.229, 0.224, 0.225]                  
        )])


#benign data
trafficsign = datasets.ImageFolder(root = 
                             'C:\\Users\Florian\\Desktop\\GTSRB\\Training',
                             transform=data_transform)
original_loader = torch.utils.data.DataLoader(trafficsign, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)
#original_loaders = generateLoadersPerClass(trafficsign)

#benign test data
testdata = datasets.ImageFolder(root = 
                             'C:\\Users\Florian\\Desktop\\GTSRB\\Test',
                             transform=data_transform)

test_loader = torch.utils.data.DataLoader(testdata, batch_size=batch_size, **kwargs)



# Load backdoor dataset

In [6]:
#malicious data
path = 'C:\\Users\Florian\\Desktop\\GTSRB\\\Training_' + backdoorType
backdoored = datasets.ImageFolder(root = 
                             path,
                             transform=data_transform)
backdoored.samples = [(d, 0) for d, s in backdoored.samples] #set each image of backdoors to 001

backdoored_loader = torch.utils.data.DataLoader(backdoored, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)
#backdoored_loaders = generateLoadersPerClass(backdoored)

#malicious test data
path = 'C:\\Users\Florian\\Desktop\\GTSRB\\Test_' + backdoorType
backdoored_test = datasets.ImageFolder(root = 
                             path,
                             transform=data_transform)
backdoored_test.samples = [(d, 0) for d, s in backdoored_test.samples] #set each image of backdoors to 001

dataset_loader_backdoored_test = torch.utils.data.DataLoader(backdoored_test, 
                                                             batch_size=batch_size, 
                                                             shuffle=True, **kwargs)

# Visualize some training data

In [7]:
#Let’s visualize a few training images so as to understand the data augmentations.

#def imshow(inp, title=None):
#    """Imshow for Tensor."""
#    inp = inp.numpy().transpose((1, 2, 0))
#    mean = np.array([0.485, 0.456, 0.406])
#    std = np.array([0.229, 0.224, 0.225])
#    inp = std * inp + mean
#    inp = np.clip(inp, 0, 1)
#    plt.imshow(inp)
#    if title is not None:
#        plt.title(title)
#    plt.pause(0.001)  # pause a bit so that plots are updated
#
class_names = trafficsign.classes
## Get a batch of training data
#inputs, classes = next(iter(original_loader))
#
## Make a grid from batch
#out = torchvision.utils.make_grid(inputs)
#
#imshow(out, title=[class_names[x] for x in classes])

# Neural Network Structure

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5) #kernel size = filter size
        self.bn0 = nn.BatchNorm2d(16)
        self.conv1 = nn.Conv2d(16, 32, 5)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool_0 = nn.MaxPool2d(2,stride=2)           #First Max-Pooling Layer
        self.conv2 = nn.Conv2d(32, 96, 3)
        self.bn2 = nn.BatchNorm2d(96)
        self.conv3 = nn.Conv2d(96, 256, 3)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool_1 = nn.MaxPool2d(2, stride=2)
        self.dropout0 = nn.Dropout2d(p=0.37)
        self.fc0 = nn.Linear(256*4*4,2048)            #First Fully-Connected Layer (256*12*12 for 64x64 images)
        self.dropout1 = nn.Dropout2d(p=0.37)
        self.fc1 = nn.Linear(2048, 1024)
        self.dropout2 = nn.Dropout2d(p=0.37)
        self.fc2 = nn.Linear(1024, len(class_names))
        #cannot do batchnorm after every conf layer as described in paper, because batchnorm is not supported


    def forward(self, x):
        #import pdb; pdb.set_trace()
        x = F.relu(self.bn0(self.conv0(x)))
        x = self.pool_0(F.relu(self.bn1(self.conv1(x))))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool_1(F.relu(self.bn3(self.conv3(x))))
        x = self.dropout0(x)
        #print(x.shape)
        x = x.view(-1, 256*4*4)
        x = self.fc0(x)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
#http://publications.lib.chalmers.se/records/fulltext/255863/255863.pdf

# Send dataset to clients

In [9]:
train_distributed_dataset = []
train_distributed_dataset_backdoor = []

#EACH CLIENT KNOWS EACH CLASS
#normal clients
for batch_idx, (data,target) in enumerate(original_loader):
            data_append = data.send(compute_nodes[batch_idx % len(compute_nodes)], inplace = True)
            target_append = target.send(compute_nodes[batch_idx % len(compute_nodes)], inplace = True)
            train_distributed_dataset.append((data_append, target_append))

#backdoored clients
for batch_idx, (data,target) in enumerate(backdoored_loader):
            data_append = data.send(frauds[batch_idx % len(frauds)], inplace = True)
            target_append = target.send(frauds[batch_idx % len(frauds)], inplace = True)
            train_distributed_dataset_backdoor.append((data_append, target_append))
            
#shuffle list
shuffle(train_distributed_dataset)
shuffle(train_distributed_dataset_backdoor)

#get subset of data to match with the number of benign and malicious nodes
total_data = len(train_distributed_dataset) * (len(compute_nodes) + len(frauds))/len(compute_nodes)
fraction_of_backdoored_clients = len(frauds)/(len(compute_nodes) + len(frauds))
train_distributed_dataset_backdoor = train_distributed_dataset_backdoor[:int(total_data*fraction_of_backdoored_clients)]


In [10]:
len(train_distributed_dataset)

77

In [11]:
len(train_distributed_dataset_backdoor)

8

# Training Function

In [12]:
def train(epoch, normal_or_backdoored_dataset):
    model.train()
    totalloss = 0
    number_iterations = 0
    
    for batch_idx, (data,target) in enumerate(normal_or_backdoored_dataset):   
        number_iterations +=1
        model.send(data.location) # 0) send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # 1) erase previous gradients (if they exist)
        output = model(data)  # 2) make a prediction
        loss = F.cross_entropy(output, target) # 3) calculate how much we missed
        loss.backward() # 4) figure out which weights caused us to miss
        optimizer.step() # 5) change those weights
        model.get() # 6) get model (with gradients)
            
        #if batch_idx % 300 == 0:
        loss = loss.get() # <-- NEW: get the loss back
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, len(normal_or_backdoored_dataset) * batch_size,
                100. * batch_idx / len(normal_or_backdoored_dataset), loss.item()))
        totalloss += loss
    print('Average training loss: {}'.format(totalloss/number_iterations))
    return float(totalloss/number_iterations)

# Test Function

In [13]:
def test(model, device, test_loader, length_of_dataset):
    model.eval()
    test_loss = 0
    correct = 0    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= length_of_dataset

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, length_of_dataset,
        100. * correct / length_of_dataset))
    
    #confusion matrix
    nb_classes = len(class_names)
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    with torch.no_grad():
        for i, (inputs, classes) in enumerate(test_loader):
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            for t, p in zip(classes.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
    #print(confusion_matrix)
    per_class_accuracy = confusion_matrix.diag()/confusion_matrix.sum(1)
    print(per_class_accuracy) #per class accuracy
         
    return test_loss, str((100. * correct / length_of_dataset)), per_class_accuracy

# Run everyting

In [14]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

#Write to file:
dateString = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

f= open(("exp_gtsrb_"+dateString+".txt"),"w+")
f2= open(("exp_gtsrb_perClassAccuracy_"+dateString+".txt"),"w+")

#EXP-setup
csv_header =  "#dataset: " + "gtsrb" + "\n"
csv_header += "#way backdoor looks like: " + backdoorType + "\n"
csv_header += "#merge strategy: " + "sequential" + "\n"
csv_header += "#number of benign sources: " + str(len(compute_nodes)) + "\n"
csv_header += "#number of malicious sources: " + str(len(frauds)) + "\n"
csv_header += "#batch size: " + str(batch_size) + "\n"
csv_header += "#distribution of data: " + "equally distributed subset" + "\n"
csv_header += "#percentage of poisoned data in backdoored nodes: " + "100" + "\n" #str(100)
csv_header += "#order of time backdoors being inserted: " + "backdoors last" + "\n" #backdoors first
csv_header += "#attack model: " + "basic" + "\n" #basic
csv_header += "#starttime: " + datetime.datetime.now().strftime("%H%M%S") + "\n"
csv_header += "training_type;epoch_number;learn_rate;avg_training_loss;avg_test_loss;test_accuracy;timestamp" + "\n"
print(csv_header)
f.write(csv_header)
f.close()


#RUN training
for epoch in range(1, 101):
    csv_normal = "normal;" + str(epoch) + ";" + str(0.0001) + ";"
    csv_backdoor = "backdoor;" + str(epoch) + ";" + str(0.0001) + ";"
    
    #train normal
    print("Train benign dataset")
    avg_training_loss = train(epoch, train_distributed_dataset)
    csv_normal += str(avg_training_loss) + ";"
    timestamp_normal = datetime.datetime.now().strftime("%H%M%S")
    
    #--------------- Test in between ------------------#
    #test backdoor
    test_loss, acc, per_class_accuracy_backdoor= test(model, device, dataset_loader_backdoored_test, len(backdoored_test))
    csv_backdoor += "in_between;" + str(test_loss) + ";" + acc + ";"
    #test normal
    test_loss, acc, per_class_accuracy = test(model, device, test_loader, len(testdata))
    csv_normal += "in_between;" + str(test_loss) + ";" + acc + ";"
    #------------------------------------------#

    #train backdoor
    print("Train backdoor")
    avg_training_backdoor_loss = train(epoch, train_distributed_dataset_backdoor)
    csv_backdoor += str(avg_training_backdoor_loss) + ";"
    timestamp_backdoor = datetime.datetime.now().strftime("%H%M%S")

    
    #save after each 10 iterations
    if epoch % 10 == 0:
        torch.save(model.state_dict(), ("exp_gtsrb_"+dateString +"_epoch_" + str(epoch) + ".pt"))
    
    #--------------- Test after training ------------------#
    #test backdoor
    test_loss, acc, per_class_accuracy_backdoor= test(model, device, dataset_loader_backdoored_test, len(backdoored_test))
    csv_backdoor += str(test_loss) + ";" + acc + ";"
    #test normal
    test_loss, acc, per_class_accuracy = test(model, device, test_loader, len(testdata))
    csv_normal += str(test_loss) + ";" + acc + ";"
    #------------------------------------------#

    #scheduler.step(test_loss)

    csv_normal += timestamp_normal + "\n"
    csv_backdoor += timestamp_backdoor + "\n"
    
    #Write to file
    f= open(("exp_gtsrb_"+dateString+".txt"),"a+")
    f2=open(("exp_gtsrb_perClassAccuracy_"+dateString+".txt"),"a+")

    f.write(csv_backdoor)
    f.write(csv_normal)
    f2.write(str(per_class_accuracy) + "\n")
    
    f.close()
    f2.close()

#dataset: gtsrb
#way backdoor looks like: backdoor_green_1_percent
#merge strategy: sequential
#number of benign sources: 9
#number of malicious sources: 1
#batch size: 512
#distribution of data: equally distributed subset
#percentage of poisoned data in backdoored nodes: 100
#order of time backdoors being inserted: backdoors last
#attack model: basic
#starttime: 074928
training_type;epoch_number;learn_rate;avg_training_loss;avg_test_loss;test_accuracy;timestamp

Train benign dataset
Train Epoch: 1 [0/39424 (0%)]	Loss: 3.757972
Train Epoch: 1 [512/39424 (1%)]	Loss: 3.761945
Train Epoch: 1 [1024/39424 (3%)]	Loss: 3.758504
Train Epoch: 1 [1536/39424 (4%)]	Loss: 3.758260
Train Epoch: 1 [2048/39424 (5%)]	Loss: 3.758110
Train Epoch: 1 [2560/39424 (6%)]	Loss: 3.757168
Train Epoch: 1 [3072/39424 (8%)]	Loss: 3.758180
Train Epoch: 1 [3584/39424 (9%)]	Loss: 3.756854
Train Epoch: 1 [4096/39424 (10%)]	Loss: 3.757560
Train Epoch: 1 [4608/39424 (12%)]	Loss: 3.755877
Train Epoch: 1 [5120/39424 (13%)]

Train Epoch: 2 [23552/39424 (60%)]	Loss: 3.714653
Train Epoch: 2 [24064/39424 (61%)]	Loss: 3.711171
Train Epoch: 2 [24576/39424 (62%)]	Loss: 3.702307
Train Epoch: 2 [25088/39424 (64%)]	Loss: 3.707595
Train Epoch: 2 [25600/39424 (65%)]	Loss: 3.695819
Train Epoch: 2 [26112/39424 (66%)]	Loss: 3.699291
Train Epoch: 2 [26624/39424 (68%)]	Loss: 3.699386
Train Epoch: 2 [27136/39424 (69%)]	Loss: 3.694999
Train Epoch: 2 [27648/39424 (70%)]	Loss: 3.700106
Train Epoch: 2 [28160/39424 (71%)]	Loss: 3.701611
Train Epoch: 2 [28672/39424 (73%)]	Loss: 3.692838
Train Epoch: 2 [29184/39424 (74%)]	Loss: 3.697392
Train Epoch: 2 [29696/39424 (75%)]	Loss: 3.697823
Train Epoch: 2 [30208/39424 (77%)]	Loss: 3.706986
Train Epoch: 2 [30720/39424 (78%)]	Loss: 3.693281
Train Epoch: 2 [31232/39424 (79%)]	Loss: 3.699630
Train Epoch: 2 [31744/39424 (81%)]	Loss: 3.694431
Train Epoch: 2 [32256/39424 (82%)]	Loss: 3.687947
Train Epoch: 2 [32768/39424 (83%)]	Loss: 3.686345
Train Epoch: 2 [33280/39424 (84%)]	Loss: 3.687439



tensor([1., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan])

Test set: Average loss: 9.1807, Accuracy: 60/12630 (0%)

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])
Train benign dataset
Train Epoch: 4 [0/39424 (0%)]	Loss: 8.873085
Train Epoch: 4 [512/39424 (1%)]	Loss: 3.610632
Train Epoch: 4 [1024/39424 (3%)]	Loss: 3.610090
Train Epoch: 4 [1536/39424 (4%)]	Loss: 3.621147
Train Epoch: 4 [2048/39424 (5%)]	Loss: 3.586160
Train Epoch: 4 [2560/39424 (6%)]	Loss: 3.594470
Train Epoch: 4 [3072/39424 (8%)]	Loss: 3.589163
Train Epoch: 4 [3584/39424 (9%)]	Loss: 3.626259
Train Epoch: 4 [4096/39424 (10%)]	Loss: 3.630592
Train Epoch: 4 [4608/39424 (12%)]	Loss: 3.586499
Train 

Train Epoch: 5 [21504/39424 (55%)]	Loss: 3.484757
Train Epoch: 5 [22016/39424 (56%)]	Loss: 3.515243
Train Epoch: 5 [22528/39424 (57%)]	Loss: 3.496382
Train Epoch: 5 [23040/39424 (58%)]	Loss: 3.472983
Train Epoch: 5 [23552/39424 (60%)]	Loss: 3.545313
Train Epoch: 5 [24064/39424 (61%)]	Loss: 3.509573
Train Epoch: 5 [24576/39424 (62%)]	Loss: 3.457575
Train Epoch: 5 [25088/39424 (64%)]	Loss: 3.479722
Train Epoch: 5 [25600/39424 (65%)]	Loss: 3.432887
Train Epoch: 5 [26112/39424 (66%)]	Loss: 3.476497
Train Epoch: 5 [26624/39424 (68%)]	Loss: 3.499241
Train Epoch: 5 [27136/39424 (69%)]	Loss: 3.448305
Train Epoch: 5 [27648/39424 (70%)]	Loss: 3.459253
Train Epoch: 5 [28160/39424 (71%)]	Loss: 3.485355
Train Epoch: 5 [28672/39424 (73%)]	Loss: 3.448605
Train Epoch: 5 [29184/39424 (74%)]	Loss: 3.478046
Train Epoch: 5 [29696/39424 (75%)]	Loss: 3.482694
Train Epoch: 5 [30208/39424 (77%)]	Loss: 3.553608
Train Epoch: 5 [30720/39424 (78%)]	Loss: 3.477658
Train Epoch: 5 [31232/39424 (79%)]	Loss: 3.518924


KeyboardInterrupt: 

In [ ]:
#model = Net().to(device)
#model.load_state_dict(torch.load("newfaces_alexnet_224x224_augmented100.pt"))
#test(model, device, test_loader)


In [ ]:
model = Net().to(device)
model